In [4]:
import VariationalBayes as vb
import LogisticGLMM_lib as logit_glmm
from VariationalBayes.SparseObjectives import Objective

import matplotlib.pyplot as plt
%matplotlib inline

#import numpy as np

import autograd
import autograd.numpy as np

import copy
from scipy import optimize

import os
import json

import time

import pickle


In [5]:
# Load data saved by stan_results_to_json.R and run_stan.R in LRVBLogitGLMM.

#analysis_name = 'simulated_data_small'
analysis_name = 'criteo_subsampled'

data_dir = os.path.join(os.environ['GIT_REPO_LOC'],
                        'LinearResponseVariationalBayes.py/Models/LogisticGLMM/data')


pickle_output_filename = os.path.join(data_dir, '%s_python_vb_results.pkl' % analysis_name)

pkl_file = open(pickle_output_filename, 'rb')
vb_results = pickle.load(pkl_file)


In [6]:
import scipy as sp
from scipy.sparse import csr_matrix

# This is not working.

# print(type(elbo_hess))
# print(elbo_hess.shape)
#sp.sparse.csr_matrix(elbo_hess)
# elbo_hess = csr_matrix(vb_results['elbo_hess'])
# print(elbo_hess)
# plt.matshow(elbo_hess[0:100, 0:100])

In [10]:
json_filename = os.path.join(data_dir, '%s_stan_dat.json' % analysis_name)

json_file = open(json_filename, 'r')
json_dat = json.load(json_file)
json_file.close()

stan_dat = json_dat['stan_dat']
#vp_base = json_dat['vp_base']

print(stan_dat.keys())
K = stan_dat['K'][0]
NObs = stan_dat['N'][0]
NG = stan_dat['NG'][0]
#N = NObs / NG
y_g_vec = np.array(stan_dat['y_group'])
y_vec = np.array(stan_dat['y'])
x_mat = np.array(stan_dat['x'])

glmm_par = logit_glmm.get_glmm_parameters(K=K, NG=NG)

# Define a class to contain prior parameters.
prior_par = logit_glmm.get_default_prior_params(K)
prior_par['beta_prior_mean'].set(np.array(stan_dat['beta_prior_mean']))

prior_par['beta_prior_info'].set(np.array(stan_dat['beta_prior_info']))

prior_par['mu_prior_mean'].set(stan_dat['mu_prior_mean'][0])
prior_par['mu_prior_info'].set(stan_dat['mu_prior_info'][0])

prior_par['tau_prior_alpha'].set(stan_dat['tau_prior_alpha'][0])
prior_par['tau_prior_beta'].set(stan_dat['tau_prior_beta'][0])

free_par_vec = glmm_par.get_free()
init_par_vec = copy.deepcopy(free_par_vec)

# Define moment parameters
moment_wrapper = logit_glmm.MomentWrapper(glmm_par)


dict_keys(['N', 'tau_prior_beta', 'y', 'beta_prior_info', 'NG', 'x', 'mu_prior_info', 'mu_prior_mean', 'beta_prior_mean', 'K', 'y_group', 'tau_prior_alpha'])


In [14]:
json_dat['glmm_fit']

{'beta_mean': [1.456, 0.0339, 0.1104, -0.1729, 0.2742],
 'beta_par': ['x.1', 'x.2', 'x.3', 'x.4', 'x.5'],
 'glmm_time': [94.9149],
 'mu_mean': [2.0591],
 'u_map': []}